#### Import dependencies 

In [1]:
# !pip install -q tensorflow==2.0.0-alpha0
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import tensorflow as tf

params = {'legend.fontsize': 'x-large',
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)
plt.style.use('ggplot')

np.random.seed(42)
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [2]:
print(tf.__version__)

2.0.0-alpha0


#### Read data 

In [3]:
X_file_path = r'data/X_train.csv'
y_file_path = r'data/y_train.csv'

In [4]:
X = pd.read_csv(X_file_path, sep=' ', header=None, lineterminator="\n").values
y = pd.read_csv(y_file_path, sep=' ', header=None, lineterminator="\n").values

In [5]:
print(X.shape)
print(y.shape)

(7767, 561)
(7767, 1)


#### Preprocess

In [6]:
# y = tf.keras.utils.to_categorical(y)[:,1:]

In [7]:
# Classes must start from 0
y -= 1

##### Shuffle data and split to train and test 

In [8]:
num_total_data = X.shape[0]
indices = np.random.permutation(num_total_data)
test_size = num_total_data // 10
training_idx, test_idx = indices[test_size:], indices[:test_size]
X_train, y_train = X[training_idx, :], y[training_idx, :]
X_test, y_test = X[test_idx, :], y[test_idx, :]

In [9]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6991, 561) (6991, 1)
(776, 561) (776, 1)


##### Convert to tf.data.Dataset  

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [11]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

#### Model

In [12]:
input_shape = X_train[0].shape
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(561,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(12, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.build()

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 4496      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 7

In [15]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
110/110 [==============================] - 1s 9ms/step - loss: 2.2465 - accuracy: 0.0299
Epoch 2/10
110/110 [==============================] - 0s 3ms/step - loss: 1.6988 - accuracy: 0.1963
Epoch 3/10
110/110 [==============================] - 0s 2ms/step - loss: 1.0574 - accuracy: 0.5179
Epoch 4/10
110/110 [==============================] - 0s 3ms/step - loss: 0.8198 - accuracy: 0.6185
Epoch 5/10
110/110 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.7182
Epoch 6/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.7599
Epoch 7/10
110/110 [==============================] - 0s 2ms/step - loss: 0.4172 - accuracy: 0.7871
Epoch 8/10
110/110 [==============================] - 0s 2ms/step - loss: 0.3795 - accuracy: 0.8098
Epoch 9/10
110/110 [==============================] - 0s 3ms/step - loss: 0.3241 - accuracy: 0.8557
Epoch 10/10
110/110 [==============================] - 0s 4ms/step - loss: 0.2896 - accuracy: 0.8863